In [1]:
import pandas as pd
import numpy as np
import datetime
import scipy.stats as stats
from scipy.stats import norm, ttest_ind
from tqdm import tqdm

### Задача  1. Оценка разных статистик
  

В лекции мы выяснили, что в эксперименте «Refactoring backend» в экспериментальной группе среднее время загрузки увеличилось, а 99% квантиль уменьшился. Проверьте значимость отличий других квантилей.

Данные эксперимента «Refactoring backend»: `2022-04-13/2022-04-13T12_df_web_logs.csv` и `2022-04-13/experiment_users.csv`. Эксперимент проводился с 2022-04-05 по 2022-04-12. Измерения времени обработки запросов считаем независимыми. При проверке используйте нормальный доверительный интервал.

In [2]:
df_web_logs = pd.read_csv('./2022-04-13T12_df_web_logs.csv')
df_exp_users = pd.read_csv('./experiment_users.csv')


print('Размер df_web_logs: ', df_web_logs.shape)
display(df_web_logs.sample(3))
print('Размер df_exp_users: ', df_exp_users.shape)
display(df_exp_users.sample(3))

Размер df_web_logs:  (2401709, 4)


,user_id,page,date,load_time
1807861,74be04,m,2022-03-27 13:31:29,76.9
1733664,461ac8,m,2022-03-25 14:38:14,80.2
91759,e1f41d,m,2022-02-06 13:53:08,76.8


Размер df_exp_users:  (23333, 2)


,user_id,pilot
20969,3f73c4,1
16352,3f42a2,1
2651,eb5e0b,0


In [3]:
#даты проведения эксперимента
date_start = datetime.datetime(2022, 4, 5)
date_end = datetime.datetime(2022, 4, 12)

In [4]:
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])
df_web_logs['date'].describe()

count                          2401709
mean     2022-03-10 05:41:51.294321920
min                2022-02-03 23:45:37
25%                2022-02-20 18:28:37
50%                2022-03-10 13:05:19
75%                2022-03-27 11:49:53
max                2022-04-13 11:59:59
Name: date, dtype: object

In [5]:
merged_df = df_web_logs[(df_web_logs['date']>=date_start)&(df_web_logs['date']<date_end)].merge(df_exp_users, on=['user_id'])

In [6]:
print(merged_df.shape)
merged_df.sample(3)

(51403, 5)


,user_id,page,date,load_time,pilot
34796,615d4c,m,2022-04-09 19:38:28,77.1,0
47811,c8fad1,m,2022-04-11 15:52:33,95.8,1
7954,e9eed9,m,2022-04-06 13:23:48,82.7,1


In [7]:
merged_df.duplicated().sum()

0

Надо проверить квантили:

- Квантиль 0.7

- Квантиль 0.74

- Квантиль 0.78 

- Квантиль 0.82 

- Квантиль 0.86 

- Квантиль 0.90

- Квантиль 0.95 

- Квантиль 0.99

- Квантиль 0.999

- Квантиль 0.9999 

In [8]:
control = merged_df[merged_df['pilot']==0].copy(deep=True)
treatment = merged_df[merged_df['pilot']==1].copy(deep=True)

In [9]:
print('Средние значения в контроле и тесте: ', control['load_time'].mean(), treatment['load_time'].mean())

Средние значения в контроле и тесте:  73.34623359478138 73.6773090568833


In [10]:
list_perc = [0.7, 0.74, 0.78, 0.82, 0.86, 0.90, 0.95, 0.99, 0.999, 0.9999]
perc_res_a_list = []
perc_res_b_list = []
for i in list_perc:
    perc_res_a_list.append(np.percentile(control['load_time'], i))
    perc_res_b_list.append(np.percentile(treatment['load_time'], i))
    
perc_res_a = dict(zip(list_perc, perc_res_a_list))
perc_res_b = dict(zip(list_perc, perc_res_b_list))

In [11]:
perc_res_a

{0.7: 37.6,
 0.74: 38.0,
 0.78: 38.4,
 0.82: 38.9,
 0.86: 39.2,
 0.9: 39.4777,
 0.95: 39.8,
 0.99: 40.0,
 0.999: 40.1,
 0.9999: 40.1}

In [12]:
perc_res_b

{0.7: 37.553599999999996,
 0.74: 38.07952,
 0.78: 38.5,
 0.82: 39.03136,
 0.86: 39.4,
 0.9: 39.7,
 0.95: 40.0,
 0.99: 40.3,
 0.999: 40.3,
 0.9999: 40.3}

In [13]:
def get_ci_bootstrap(b_metrics:np.array, pe_metric: float, alpha: float=0.05):
    '''Ф-я для оценки доверит интервала для квантиля
    
    input:
    b_metrics - значение метрики, полученные с помощью бутстрепа
    pe_metric - точечная оценка метрики
    alpha - уровень значимости

    output:
    ci_left, ci_right границы доверит интервала
    '''

    z = norm.ppf(1-alpha/2)
    se = np.std(b_metrics)
    ci_left, ci_right = pe_metric - z * se, pe_metric + z * se

    return ci_left, ci_right

In [14]:
n = 2000
alpha = 0.05
values_a = control['load_time'].values
values_b = treatment['load_time'].values
quantiles_arr = np.array(list_perc)


In [15]:
results = []

for q in quantiles_arr:
    pe = np.quantile(values_b, q) - np.quantile(values_a, q)
    bootstrap_values_a = np.random.choice(values_a, (n, len(values_a)), True)
    bootstrap_metrics_a = np.quantile(bootstrap_values_a, q, axis=1)
    bootstrap_values_b = np.random.choice(values_b, (n, len(values_b)), True)
    bootstrap_metrics_b = np.quantile(bootstrap_values_b, q, axis=1)
    stats_btsrp = bootstrap_metrics_b - bootstrap_metrics_a
    ci_left, ci_right = get_ci_bootstrap(stats_btsrp, pe, alpha)
    results.append({
    'quantile':q,
    'ci_left':ci_left,
    'ci_right':ci_right,
    'significant':not (ci_left < 0 < ci_right)
    })

In [16]:
pd.DataFrame(results)

,quantile,ci_left,ci_right,significant
0,0.7000,-0.382516,0.182516,False
1,0.7400,-0.375127,0.175127,False
2,0.7800,-0.378688,0.178688,False
3,0.8200,-0.294740,0.294740,False
4,0.8600,-0.312114,0.312114,False
5,0.9000,-0.313314,0.313314,False
6,0.9500,-0.637112,0.167112,False
7,0.9900,-0.613417,1.213417,False
8,0.9990,-1169.970289,1873.528689,False
9,0.9999,-479.384746,240.072386,False


### Задача 2. Функция оценки эксперимента с помощью bootstrap

Реализуйте функцию `run_bootstrap`.

Обратите внимание, что на вход функции подаются не исходные значения метрики, а множество значений статистики теста, посчитанные на бутстрепных выборках. Это позволит нам детерминировано протестировать правильноcть решения. Самостоятельно бутстрепить данные внутри функции run_bootstrap не нужно.

Пример реализации функции для вычисления bootstrap_metrics


In [17]:
def generate_bootstrap_metrics(data_one, data_two, bootstrap_iter, bootstrap_agg_func):
    """Генерирует значения метрики, полученные с помощью бутстрепа.

    :param data_one, data_two (np.array): значения метрик в группах.
    :param design (Design): объект с данными, описывающий параметры эксперимента
    :param bootstrap_iter (int): количество итераций бутстрепа.
    :param bootstrap_agg_func (str): метрика эксперимента.
        Возможные значения ['mean', 'quantile 95'].
    :return bootstrap_metrics, pe_metric:
        bootstrap_metrics (np.array) - множество значений статистики теста,
            посчитанные на бутстрепных выборках.
        pe_metric (float) - значение статистики теста посчитанное по исходным данным.
    """
    bootstrap_data_one = np.random.choice(data_one, (len(data_one), bootstrap_iter))
    bootstrap_data_two = np.random.choice(data_two, (len(data_two), bootstrap_iter))
    if bootstrap_agg_func == 'mean':
        bootstrap_metrics = (
            bootstrap_data_two.mean(axis=0) - bootstrap_data_one.mean(axis=0)
        )
        pe_metric = data_two.mean() - data_one.mean()
        return bootstrap_metrics, pe_metric
    elif bootstrap_agg_func == 'quantile 95':
        q = 0.95
        bootstrap_metrics = (
            np.quantile(bootstrap_data_two, q, axis=0)
            - np.quantile(bootstrap_data_one, q, axis=0)
        )
        pe_metric = np.quantile(data_two, q) - np.quantile(data_one, q)
        return bootstrap_metrics, pe_metric
    else:
        raise ValueError('Неверное значение bootstrap_agg_func')

In [21]:
import numpy as np
from scipy import stats


def run_bootstrap(bootstrap_metrics, pe_metric, alpha, bootstrap_ci_type):
    """Строит доверительный интервал и проверяет значимость отличий с помощью бутстрепа.
    
    :param bootstrap_metrics (np.array): множество значений статистики теста,
        посчитанные на бутстрепных выборках.
    :param pe_metric (float): значение статистики теста посчитанное по исходным данным.
    :param alpha (float): уровень значимости.
    :param bootstrap_ci_type (str): способ построения доверительного интервала.
        Возможные значения ['normal', 'percentile', 'pivotal'].
    :return ci, pvalue:
        ci [float, float] - границы доверительного интервала.
        pvalue (float) - 0 если есть статистически значимые отличия, иначе 1. Настоящее
        pvalue для произвольного способа построения доверительного интервала с помощью
        бутстрепа вычислить не тривиально. Будем использовать краевые значения 0 и 1.
    """
    
    if bootstrap_ci_type == 'normal':
        z = norm.ppf(1-alpha/2)
        se = np.std(bootstrap_metrics)
        ci_left, ci_right = pe_metric - z * se, pe_metric + z * se
    
    elif bootstrap_ci_type == 'percentile':
        ci_left, ci_right = np.quantile(bootstrap_metrics, [alpha / 2, 1 - alpha / 2])

    elif bootstrap_ci_type == 'pivotal':
        ci_left, ci_right = 2 * pe_metric - np.quantile(bootstrap_metrics, [1 - alpha / 2, alpha / 2])
    else:
        raise ValueError('Неверное значение bootstrap_ci_type')
        
    if not (ci_left < 0 < ci_right):
        pvalue = float(0)
    else:
        pvalue = float(1)
    return [ci_left, ci_right], pvalue

        

In [22]:
#Тестовый пример 1
data_one, data_two = np.array([1, 3]), np.array([5, 7])
bootstrap_iter = 10
bootstrap_agg_func = 'mean'
bootstrap_metrics, pe_metric = generate_bootstrap_metrics(
    data_one, data_two, bootstrap_iter, bootstrap_agg_func
)


print(bootstrap_metrics)
print(pe_metric)

run_bootstrap(bootstrap_metrics, pe_metric, alpha=0.05, bootstrap_ci_type='normal')

[4. 2. 4. 5. 5. 5. 5. 3. 5. 5.]
4.0


([2.030260573349595, 5.9697394266504045], 0.0)

In [23]:
#Тестовый пример 2
bootstrap_metrics = np.arange(-90, 910)
pe_metric = 600.
alpha = 0.05
bootstrap_ci_types = ['normal', 'percentile', 'pivotal']
for bootstrap_ci_type in bootstrap_ci_types:
    ci, pvalue = run_bootstrap(bootstrap_metrics, pe_metric, alpha, bootstrap_ci_type)
    print(bootstrap_ci_type)
    print(f'ci = {np.array(ci).round()}, pvalue = {pvalue}')

normal
ci = [  34. 1166.], pvalue = 0.0
percentile
ci = [-65. 884.], pvalue = 1.0
pivotal
ci = [ 316. 1265.], pvalue = 0.0
